# Exploring putative circular DNA in parasite and related species

This notebook showcases a second use case for the circular DNA-finding workflow. We applied the Nextflow pipeline to a set of parasitic (and related non-parasitic) species to explore potential circular DNAs. This notebook presents the uses of the `GenomeInfo` class to explore annotations and coverage relatively easily.

If you'd like to run this, to properly import the `GenomeInfo` class, either add the path to `scripts` into your `$PATH`, or in VSCode change the user setting `Notebook File Root` to `${workspaceFolder}`.

This notebook also uses the `parasite_annotation` conda environment.

We generated the BAM files and coverage files using the Nextflow pipeline with the following command:

```bash
nextflow run main.nf --samples inputs/parasite_species_samplesheet.csv --outdir results/parasite_species --threads 16 -resume
```

Then we moved the BAM files and coverage files into separate subdirectories to make it easier to work with them.

I also quickly indexed the BAM files using `samtools index` to make it easier to work with them.

In [1]:
import glob
import os

import polars as pl

# Import the class used to handle the genome information
from scripts.genomeinfo import GenomeInfo

In [2]:
GFF_DIR = "inputs/parasite_species/gffs"
BAM_DIR = "results/parasite_species/bam_files"
COVERAGE_DIR = "results/parasite_species/coverage_files"

# Get file prefixes from the BAM files
file_prefixes = [os.path.basename(f).split(".large_inserts.bam")[0] for f in glob.glob(f"{BAM_DIR}/*.large_inserts.bam")]

The first step is to load in the required (and optional) files to create a `GenomeInfo` object.

In [29]:
genome_infos = []

for prefix in file_prefixes:
    print(f"Processing {prefix}...")
    genome_info = GenomeInfo(
        large_insert_bam_loc = f"{BAM_DIR}/{prefix}.large_inserts.bam",
        large_insert_bai_loc = f"{BAM_DIR}/{prefix}.large_inserts.bam.bai",
        coverage_loc = f"{COVERAGE_DIR}/{prefix}.filtered_coverage.txt",
        gff_loc = f"{GFF_DIR}/{prefix.split('_vs_')[0]}_genomic.gff.gz",
        genome_metadata = {"prefix": prefix}
    )
    genome_info.load_bam()
    genome_info.load_coverage()
    if os.path.exists(genome_info.gff_loc):
        genome_info.load_gff()
    genome_infos.append(genome_info)

Processing GCA_001447435.1_T6_ISS34_r1.0_vs_SRR14041386...
Processing GCA_020844145.1_ASM2084414v1_vs_SRR10821146...
Processing GCF_013339725.1_ASM1333972v1_vs_SRR14626932...
Processing GCF_030504385.1_Musca_domestica.polishedcontigs.V.1.1_vs_SRR650029...
Processing GCA_001447585.1_T3_ISS120_r1.0_vs_SRR14041383...
Processing GCF_000313135.1_Acastellanii.strNEFF_v1_vs_SRR15928354...
Processing GCA_001447565.2_T2_ISS10_r1.1_vs_SRR14041382...
Processing GCF_000181795.1_Trichinella_spiralis-3.7.1_vs_SRR14041381...
Processing GCA_037355805.1_I.ric_WGA_PrF_JU_vs_SRR18810692...
Processing GCF_013339725.1_ASM1333972v1_vs_SRR11263447...
Processing GCA_020844145.1_ASM2084414v1_vs_SRR15371646...
Processing GCF_000699445.3_UoM_Shae.V3_vs_SRR433860...
Processing GCA_013358835.2_BIME_Iper_1.3_vs_SRR25318387...
Processing GCF_030504385.1_Musca_domestica.polishedcontigs.V.1.1_vs_SRR4217591...
Processing GCF_000648675.2_Clec_2.1_vs_SRR6985627...
Processing GCA_000181055.3_Rhodnius_prolixus-3.0.3_vs_SRR

Create and write the analyzed files to the `results/parasite_species/insert_filtered_results/` directory. This step creates the following files for each genome/SRA pair:
1. [genome]_vs_[sra_accession]_deduped_insert_summary.csv: the output of the insert size analysis, with the genomic ranges and counts for each insert size with a z score over the threshold (default 5 in this case). It also is filtered to only include insert sizes <= 100kb.
2. [genome]_vs_[sra_accession]_insert_filtered_coverage.csv: the coverage data filtered to only include the insert sizes found in the insert summary file.
3. [genome]_vs_[sra_accession]_insert_filtered_gff.csv: the annotation data filtered to only include the insert sizes found in the insert summary file.
4. [genome]_vs_[sra_accession]_merged_bam_coverage.csv: the merged coverage and insert size data produced after finding outlier inserts and positions.

The GFF outputs may or may not exist depending on the genome, as not all genomes have annotations available.

This is just one way to use `GenomeInfo`: if you want to tweak the parameters or look at the insert data without adding genomic ranges around them, please do so!

In [ ]:
genome_params = {"maximum_insert_size_threshold": 100000,
                 "genomic_range_width_bp": 10000,
                 "z_score_threshold": 5}

peak_dict = {}

# Check if the output directory exists, if not create it
OUTPUT_DIR = "results/parasite_species/insert_filtered_results"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

for genome in genome_infos:
    print(f"Starting work on {genome.genome_metadata['prefix']}")
    peak_dict[genome.genome_metadata["prefix"]] = {}
    # Generate the deduplicated insert summary for the genome and reads
    deduped_output = f"{OUTPUT_DIR}/{genome.genome_metadata['prefix']}_deduped_insert_summary.csv"
    if not os.path.exists(deduped_output):
        insert_summary = genome.generate_insert_summary(maximum_size_threshold=genome_params["maximum_insert_size_threshold"])
        peak_dict[genome.genome_metadata["prefix"]]["insert_summary"] = insert_summary
        insert_range = genome.generate_insert_range(insert_summary, width_bp=genome_params["genomic_range_width_bp"])
        peak_dict[genome.genome_metadata["prefix"]]["insert_range"] = insert_range
        deduped = genome.deduplicate_insert_summary(insert_range, z_score_threshold=genome_params["z_score_threshold"])
        peak_dict[genome.genome_metadata["prefix"]]["deduped"] = deduped
        deduped.write_csv(
            f"{OUTPUT_DIR}/{genome.genome_metadata['prefix']}_deduped_insert_summary.csv"
        )
    else:
        deduped = pl.read_csv(deduped_output)
        peak_dict[genome.genome_metadata["prefix"]]["deduped"] = deduped

    # Filter the coverage file to only positions within detected mapped distance sizes
    filtering_pos = genome.generate_filtering_positions(deduped)
    
    cov_output = f"{OUTPUT_DIR}/{genome.genome_metadata['prefix']}_insert_filtered_coverage.csv"
    if not os.path.exists(cov_output):
        filtered_coverage = genome.filter_coverage(filtering_pos)
        peak_dict[genome.genome_metadata["prefix"]]["filtering_pos"] = filtering_pos
        filtered_coverage.write_csv(
            f"{OUTPUT_DIR}/{genome.genome_metadata['prefix']}_insert_filtered_coverage.csv"
        )
        peak_dict[genome.genome_metadata["prefix"]]["coverage"] = filtered_coverage
    else:
        # Check if filtering_pos is empty, if not load file
        if len(filtering_pos) >= 1:
            filtered_coverage = pl.read_csv(cov_output)
            peak_dict[genome.genome_metadata["prefix"]]["filtering_pos"] = filtering_pos
            peak_dict[genome.genome_metadata["prefix"]]["coverage"] = filtered_coverage

    # If there's a GFF file, filter it to only include positions within detected mapped distance sizes
    if genome.gff is not None:
        gff_output = f"{OUTPUT_DIR}/{genome.genome_metadata['prefix']}_insert_filtered_gff.csv"
        if not os.path.exists(gff_output):
            filtered_gff = genome.filter_gff(filtering_pos)
            peak_dict[genome.genome_metadata["prefix"]]["filtered_gff"] = filtered_gff
            # Check if the filtered gff is empty, if not write it out
            if not filtered_gff.is_empty():
                filtered_gff.write_csv(
                    f"{OUTPUT_DIR}/{genome.genome_metadata['prefix']}_insert_filtered_gff.csv"
                )
        else:
            filtered_gff = pl.read_csv(gff_output, ignore_errors=True)
            peak_dict[genome.genome_metadata["prefix"]]["filtered_gff"] = filtered_gff

    # Filter the BAM file and merge together with the filtered coverage, only if the filtering_pos is not empty
    # If genome prefix has "Musca", skip - too large to process in a reasonable amount of time
    if len(filtering_pos) >= 1:
        if "Musca" not in genome.genome_metadata["prefix"]:
            bam_cov_output = f"{OUTPUT_DIR}/{genome.genome_metadata['prefix']}_merged_bam_coverage.csv"
            if not os.path.exists(bam_cov_output):
                bam_data = []
                for read in genome.bam.fetch():
                    # Check if read.reference_name is in filtering_pos keys, skip if not
                    if read.reference_name not in filtering_pos.keys():
                        continue
                    else:
                        chr_filtering_pos = filtering_pos[read.reference_name]
                        if read.reference_start in chr_filtering_pos:
                            appendable_data = {
                                "query_name": read.query_name,
                                "chromosome": read.reference_name,
                                "position": read.reference_start,
                                "reference_end": read.reference_end,
                                "length": read.template_length,
                                "cigar": read.cigarstring,
                            }
                            bam_data.append(appendable_data)
                bam_df = pl.DataFrame(bam_data)
                # Check if filtered_coverage is empty, if not merge it with the bam_df
                if not filtered_coverage.is_empty():
                    merged_df = bam_df.join(filtered_coverage, on=["chromosome", "position"])
                merged_df.write_csv(
                        f"{OUTPUT_DIR}/{genome.genome_metadata['prefix']}_merged_bam_coverage.csv"
                    )
            else:
                merged_df = pl.read_csv(bam_cov_output)
                peak_dict[genome.genome_metadata["prefix"]]["merged_bam_coverage"] = merged_df
    
    print(f"Finished working on {genome.genome_metadata['prefix']}")